# 01 | Transform the Data
# Introduction
In this notebook, we will deal with the data transformation steps required to proceed with our analysis. This process will be reproduced as .py files later on.

# Goals:
Targeted objectives in this notebook ar checked as follows:

- [ ] Import the raw data
  -  [ ] SIPRI dataset and capitals coordinates
- [ ]  Store the raw data
- [x] Prepare the data
  - [x] Clean each individual table
  - [x] Store the transformed dataset
- Combine both datasets
  - Check if the merging column values match
- Store the final output

# Set up our working environment

In [1]:
# Import required libraries
import pandas as pd
import os

In [2]:
# Create directory folders to store our data
dirname = os.getcwd()

raw_data = f"{dirname}/data/raw/"
transformed_data = f"{dirname}/data/transformed/"
refined_data = f"{dirname}/data/refined/"

paths = [raw_data, transformed_data, refined_data]

for path in paths:
    if not os.path.exists(path):
        os.makedirs(path)

# Data Transformation | SIPRI dataset
Let's start by preparing our main dataset for data analysis.

## Import raw data
We'll first import the raw data that we stored during the previous step. Because we have three indivual tables that must turn into one, we must establish a few steps:
- Import a single table;
- Analyse the required transformation steps we need;
- Apply the transformations;
- Loop through the 3 tables and pass the same steps (if possible);

We can start by using table 5 (Constant US$ (2022))

In [5]:
sipri_raw = pd.read_csv(f"{raw_data}sipri_data_raw_tb_5.csv")
sipri_raw

,"Military expenditure by country, in millions of US$ at current prices and exchange rates, 1948-2023 © SIPRI 2023",Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 68,Unnamed: 69,Unnamed: 70,Unnamed: 71,Unnamed: 72,Unnamed: 73,Unnamed: 74,Unnamed: 75,Unnamed: 76,Unnamed: 77
0,"Figures are in US $m., in current prices, conv...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Figures in blue are SIPRI estimates. Figures i...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,""". ."" = data unavailable. ""xxx"" = country did ...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Country,Notes,1948,1949,1950,1951,1952,1953,1954,1955,...,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
193,Syria,§,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
194,Türkiye,‖,...,197.68186020052622,212.97020550380432,231.81397994737964,257.7686126715495,294.0339899025812,332.0770817037616,382.9197184100121,...,17576.538470505897,15668.749999999998,17827.702150796664,17822.738263164494,19648.69382385138,20436.917121238785,17478.41368526983,15567.410029425082,10779.896284618242,15827.853255045886
195,United Arab Emirates,§,...,...,...,...,...,...,...,...,...,22755.071477195375,...,...,...,...,...,...,...,...,...
196,"Yemen, North",§,...,...,...,...,...,...,...,...,...,xxx,xxx,xxx,xxx,xxx,xxx,xxx,xxx,xxx,xxx
